In [1]:
import pandas as pd
import random
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df

## Using Gutenberg Poetry dataset

In [ ]:
!head -n5 './data/gutenberg-poetry-v001.ndjson'

In [ ]:
p = 0.005  # 0.5% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
    './data/gutenberg-poetry-v001.ndjson',
    names=['text'], 
    skiprows=lambda i: i>0 and random.random() > p,
    sep='\n',
#    columns=['text']
)

df = json_normalize(df['text'].apply(json.loads))
text_col = 's'
print(df.shape)

## Using Lyricsfreak.com dataset (from kaggle, under CreativeCommons)

In [3]:
!head -n10 ./data/songdata.csv

artist,song,link,text
ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  


In [4]:
p = 0.05  # 5% of the lines
df = pd.read_csv(
    './data/songdata.csv',
    skiprows=lambda i: i>0 and random.random() > p,
)

df['text'] = df['text'].str.replace('\n', '')
text_col = 'text'
print(df.shape)

(2900, 4)


In [5]:
df.head()

,artist,song,link,text
0,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please Touch me gently ..."
1,ABBA,I Let The Music Speak,/a/abba/i+let+the+music+speak_20003002.html,"I'm hearing images, I'm seeing songs No poet ..."
2,ABBA,Like An Angel Passing Through My Room,/a/abba/like+an+angel+passing+through+my+room_...,Long awaited darkness falls Casting shadows o...
3,ABBA,"My Love, My Life",/a/abba/my+love+my+life_20002559.html,I've seen it on your face Tells me more than ...
4,ABBA,Put On Your White Sombrero,/a/abba/put+on+your+white+sombrero_20003024.html,Put on your white sombrero Saddle your horse ...


In [9]:
from bert_serving.client import BertClient

bc = BertClient(ignore_all_checks=False)

In [10]:
%%time
encodings = bc.encode(df[text_col].values.tolist())

/usr/local/lib/python3.7/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


CPU times: user 31.4 ms, sys: 19.1 ms, total: 50.5 ms
Wall time: 3min 58s


In [15]:
encodings.shape

(2900, 768)

In [12]:
#entry = 'The sky is blue and warm'
#entry = 'Nothing matters more !'
#entry = 'Shadow of my dream'
#entry = 'How are you doing ?'
#entry = 'Have a good day !'
#entry = 'Walk into the moon'
#entry = 'Come on stranger'
#entry = 'A world full of fantasy'
entry = 'Stars are falling'

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

enc_test = bc.encode([entry])

scores = cosine_similarity(enc_test, encodings)

top = 7
idxs = np.argsort(scores)[0][-top:]
idxs = np.flip(idxs)

In [18]:
## Show similar lyrics/songs
print(scores[0][idxs])

df_tmp = pd.concat([df.iloc[idxs].reset_index(), pd.DataFrame(scores[0][idxs], columns=['score'])], axis=1)
df_tmp.head(20)

[0.8085018  0.8004481  0.79822963 0.79820204 0.79628587 0.7938714
 0.7882048 ]


,index,artist,song,link,text,score
0,1277,Beautiful South,Dream A Little Dream,/b/beautiful+south/dream+a+little+dream_100271...,Stars shining bright above you Night breezes ...,0.808502
1,62,Bette Midler,Mele Kalikimaka,/b/bette+midler/mele+kalikimaka_20529217.html,jingle bells upon a steel guitar thru the clo...,0.800448
2,1037,U2,Promenade,/u/u2/promenade_20141533.html,Earth sky sea and rain Is she coming back aga...,0.798230
3,2365,Orphaned Land,The Beloved's Cry,/o/orphaned+land/the+beloveds+cry_20157154.html,"My heart flies on her wings Now I'm crying, t...",0.798202
4,1411,Celine Dion,If Only You Could See Me Now,/c/celine+dion/if+only+you+could+see+me+now_20...,"(Walter Afanasieff, John Bettis) Here they ...",0.796286
5,1603,Electric Light Orchestra,New World Rising,/e/electric+light+orchestra/new+world+rising_2...,"New dawn, smiling down the old blue world Can...",0.793871
6,276,Emmylou Harris,I Still Miss Someone,/e/emmylou+harris/i+still+miss+someone_2005004...,At my door the leaves are falling The cold wi...,0.788205
